# ResNet builder

### This notebook will help you to understand all the APIs

In [1]:
from tensorflow.keras.optimizers import Adam
from core.data_loader import DataLoader
from core.resnet import *
from utils.callbacks import callbacks
from utils.misc_utils import *


In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Limiting Tensorflow debugging message

depth = 56 # No of layers to be used to build ResNet model
batch_size = 32
epochs = 50
# TF GPU memory graph
limit_gpu() # Useful for local system

[INFO]...  1 Physical GPUs, 1 Logical GPUs


### Step 1: Data loader

The DataLoader() support two data configration:
1. Data is in seprate directory for training and validation data
2. Data is in common directory 


In [4]:
# Building a data loader
data_loader = DataLoader(validation_split=0.2)

In [ ]:
# If data is in seprate directory
train_generator = data_loader.from_dir(directory='path/to/training_dataset')
val_generator = data_loader.from_dir(directory='path/to/validation_dataset')

In [ ]:
# If data is in common directory
"""
DataLoader.from_common_dir() return a data generator & xtest, ytest used for validation data.
DataLoader also store lots static values like num_classes, train_len(no of training data points),
val_len(no of validation data points, etc which will be needed)

"""
data_generator,xtest, ytest = data_loader.from_common_dir(directory='path/to/dataset',
                                            batch_size=batch_size)

### Step 2: Building ResNetV2 model

Four types of confrigation are supported:

1. Build a complete ResNetV2 model with n no of residual layers with Fully connected layers included
2. Add n no of ResNetV2 layers in your already created model
2. Fine Tune ResNetV2 with fully connected layer included
3. Fine Tune ResNetV2 with fully connected layer not included

In [ ]:
# 1. Build a complete ResNetV2 model with n no of residual layers with Fully connected layers included
model = build_resnet_model(
    input_shape=(32, 32, 3),
    depth=depth,
    num_classes=data_loader.num_classes
)

In [ ]:
# 2. Add n no of ResNetV2 layers in your already created model
"""
Extra care have to taken in this method for matching the tensor shape of 
itermediate resent layer
"""
inputs = tf.keras.layers.Input(shape=input_shape)
X = tf.keras.layers.Conv2D(64,
                        padding='same',
                        kernel_regularizer=l2(1e-4))(inputs)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Activation('relu')(X)

res_layer = build_resnet_layer(inputs=X, num_filters_in=64, depth=5)

y = Flatten()(res_layer)
y = Dense(512, activation='relu')(y)
y = BatchNormalization()(y)
y = Dropout(0.5)(y)

outputs = Dense(num_classes,
                    activation='softmax')(y)

# Instantiate model.
model = Model(inputs=inputs, outputs=outputs)
return model

In [ ]:
# 3. Fine Tune ResNetV2 with fully connected layer included
"""
Use this if you wish to fine tune on pretrained imagenet weights.
A FC layer have already appened here. if you wish to used your desired 
FC layer then used core.resnet.build_resnet_pretrained_customized()

"""

model = build_resnet_pretrained(input_shape=(224,224,3))

In [ ]:
# 4. Fine Tune ResNetV2 with fully connected layer not included
X = build_resnet_pretrained_customized(input_shape = (224,224,3)
                                       
# Append FC layer
y = X.output
y = Flatten()(y)
y = Dense(10, activation='softmax')(y)

# Combining base model FC head model
model = Model(inputs=x.input, outputs=y)

# Freezing weights

for layer in base.layers:
    layer.trainable = False

### Step 3 (Optional): Use callbacks

In [ ]:
"""
It is optional though advisable to use them
"""
callbacks = callbacks(
    save_path='./assets/weights/exp2',
    depth=depth
)

### Step 4: Compiling model

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(amsgrad=True, decay=0.001/epochs),
    metrics=['accuracy']
)

### Step 5: Starting Training

In [ ]:
history = model.fit(
    x=data_generator,
    epochs=epochs,
    steps_per_epoch=int(data_loader.train_len/batch_size),
    callbacks=callbacks,
    validation_data=(xtest, ytest),
    validation_steps=int(data_loader.val_len/batch_size)
)

In [ ]:
# Visualize training
visualize(
    history=history.history,
    save_dir='./assets/logs'
)